<a href="https://colab.research.google.com/github/S-Kaito/s-kaito.github.io/blob/master/notebook/seminar/2020_0609.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import struct
import numpy as np
import codecs
import math
import itertools
from PIL import Image
import numpy as np

def read_picture(path):
    """
    Read picture from file

    Parameter
    ---------
    path: string
        file path
    
    return
    ------
    size: (width: int, height: int)
        picture size
    max_n: int
        max value
    data: NdArray<NdArray<int>>
        picture data
    """
    
    # 画像を開く(Header情報用)
    infile = codecs.open(path, 'r', "utf-8", "ignore")
    # 画像を開く(画像データ用)
    img = np.array(Image.open(path))

    infile.readline()
    infile.readline()
    width, height = map(int, infile.readline().split())
    max_n = int(infile.readline())

    infile.close()
    return (width, height), max_n, img

def write_picture(path, size, n, data):
    """
    Write picture to file
    Parameter
    ---------
    path: string
        file path
    size: (width: int, height: int)
        picture size
    n: int
        max value
    data: Array<Array<[r: int, g: int, b: int]>>
        picture data
    """

    # 画像を書き出す
    pilImg = Image.fromarray(np.uint8(data.reshape((size[0], size[1]))))
    pilImg.save(path)

def parse_gray_scale(data):
    # NTSC形式用の配列
    value = np.array([
        [0.299, 0.587, 0.114],
        [0.596, -0.274, -0.322],
        [0.212, -0.523, 0.311]
    ])

    data = data.reshape((65536, 3))

    #行列の掛け算を行い、そのうちのYのみで配列を生成する
    res = [math.floor(np.dot(value, d)[0]) for d in data]

    return np.array(res).reshape((256, 256))

def binalize(img, threshold = 128):
    return np.array([[(255 if i > threshold else 0) for i in im] for im in img])

def binalize_avg(img):
    average = np.average(img)
    return np.array([[(255 if i > average else 0) for i in im] for im in img])

def generate_dither(img, type="bayer"):
    matrix = [8]
    size = (1, 1)
    if type == "bayer":
        matrix = np.array([ [ 0, 8, 2,10],
                            [12, 4,14, 6],
                            [ 3,11, 1, 9],
                            [15, 7,13, 5]]) * 16
        size = (4, 4)
    elif type == "halftone":
        matrix = np.array([ [10, 4, 6, 8],
                            [12, 0, 2,14],
                            [ 7, 9,11, 5],
                            [ 3,15,13, 1]]) * 16
        size = (4, 4)
    elif type == "intermediate_emphasis":
        matrix = np.array([ [12, 4, 8,14],
                            [11, 0, 2, 6],
                            [ 7, 3, 1,10],
                            [15, 9, 5,13]]) * 16
        size = (4, 4)
    elif type == "original":
        matrix = np.array([ [ 9, 4,13, 6],
                            [ 2, 8, 7,12],
                            [10, 6, 9, 5],
                            [ 7,11, 3, 8]]) * 16
        size = (4, 4)
    return np.array([[(255 if img[i][j] > matrix[i % size[0]][j % size[1]] else 0) for j in range(img.shape[0])] for i in range(img.shape[1])])

In [2]:
PATH = "girl_noise.ppm"

size, n, img = read_picture(PATH)

# 画像をグレイスケールに変換
img = parse_gray_scale(img)

# 固定閾値法で2値化
img_b = binalize(img)
write_picture("girl_b.pgm", size, n, img_b)

# 平均閾値法で2値化
img_a = binalize_avg(img)
write_picture("girl_a.pgm", size, n, img_a)

# Bayerマトリクスで疑似中間調画像の生成
img_d = generate_dither(img)
write_picture("girl_d.pgm", size, n, img_d)

# ハーフトーンマトリクスで疑似中間調画像の生成(発展課題02)
img_dh = generate_dither(img, type="halftone")
write_picture("girl_dh.pgm", size, n, img_dh)

# 中間調強調マトリクスで疑似中間調画像の生成(発展課題02)
img_di = generate_dither(img, type="intermediate_emphasis")
write_picture("girl_di.pgm", size, n, img_di)

# オリジナルのマトリクスで疑似中間調画像の生成
img_do = generate_dither(img, type="original")
write_picture("girl_do.pgm", size, n, img_do)

print("Finish!!")

Finish!!
